In [7]:
import os
import xml.etree.ElementTree as ET
import cv2

# Function to parse XML annotation files
def parse_annotation(annotation_file):
    tree = ET.parse(annotation_file)
    root = tree.getroot()

    filename = root.find('filename').text
    size = root.find('size')
    image_width = int(size.find('width').text)
    image_height = int(size.find('height').text)

    objects = []
    for obj in root.findall('object'):
        obj_name = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        objects.append({
            'name': obj_name,
            'bbox': [xmin, ymin, xmax, ymax]
        })

    return filename, image_width, image_height, objects

# Define label mapping
label_mapping = {
    'D00': 'Longitudinal Crack',
    'D10': 'Transverse Crack',
    'D20': 'Alligator Crack',
    'D40': 'Pothole'
}

# Function to map object names to labels
def map_labels(objects):
    labels = []
    for obj in objects:
        obj_name = obj['name']
        if obj_name in label_mapping:
            labels.append(label_mapping[obj_name])
    return labels

# Path to the directory containing annotation files
annotations_dir = r'/Users/boomika/Desktop/xmls'

# Path to the directory containing images
images_dir = r'/Users/boomika/Desktop/images'

# Define paths for saving annotated images and labels
train_dir = r'/Users/boomika/Desktop/train'
val_dir = r'/Users/boomika/Desktop/validation'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# List to store parsed annotations
annotations = []

# Parse each annotation file in the directory
for annotation_file in os.listdir(annotations_dir):
    annotation_path = os.path.join(annotations_dir, annotation_file)
    filename, image_width, image_height, objects = parse_annotation(annotation_path)
    image_path = os.path.join(images_dir, filename)
    annotations.append({
        'image_path': image_path,
        'image_width': image_width,
        'image_height': image_height,
        'objects': objects
    })

# Save images with bounding boxes and create labels
for idx, annotation in enumerate(annotations):
    image = cv2.imread(annotation['image_path'])
    
    # Draw bounding boxes on the image
    for obj in annotation['objects']:
        xmin, ymin, xmax, ymax = obj['bbox']
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    
    # Map labels for the current annotation
    labels = map_labels(annotation['objects'])
    
    # Determine destination directory based on index for splitting between train and validation
    if idx % 5 == 0:
        dest_dir = val_dir
    else:
        dest_dir = train_dir
    
    # Save annotated image
    dest_image_path = os.path.join(dest_dir, os.path.basename(annotation['image_path']))
    cv2.imwrite(dest_image_path, image)

    # Save labels to text file
    label_filename = os.path.splitext(os.path.basename(annotation['image_path']))[0] + '.txt'
    dest_label_path = os.path.join(dest_dir, label_filename)
    with open(dest_label_path, 'w') as f:
        for label in labels:
            f.write(label + '\n')

    print(f"Image {idx + 1} processed. Labels: {labels}")

print("Images with bounding boxes and labels saved successfully in train and validation folders.")


Image 1 processed. Labels: ['Alligator Crack', 'Longitudinal Crack']
Image 2 processed. Labels: []
Image 3 processed. Labels: ['Alligator Crack', 'Pothole', 'Pothole']
Image 4 processed. Labels: []
Image 5 processed. Labels: []
Image 6 processed. Labels: ['Pothole', 'Pothole', 'Pothole', 'Pothole', 'Pothole', 'Pothole', 'Pothole']
Image 7 processed. Labels: ['Alligator Crack']
Image 8 processed. Labels: []
Image 9 processed. Labels: ['Longitudinal Crack', 'Pothole', 'Pothole', 'Pothole', 'Pothole', 'Pothole']
Image 10 processed. Labels: ['Longitudinal Crack', 'Alligator Crack', 'Longitudinal Crack']
Image 11 processed. Labels: []
Image 12 processed. Labels: []
Image 13 processed. Labels: ['Alligator Crack']
Image 14 processed. Labels: ['Alligator Crack', 'Longitudinal Crack']
Image 15 processed. Labels: ['Alligator Crack']
Image 16 processed. Labels: ['Alligator Crack']
Image 17 processed. Labels: ['Longitudinal Crack']
Image 18 processed. Labels: []
Image 19 processed. Labels: []
Imag